In [1]:
from dash import Dash, dcc, html, Input, Output
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
import country_converter as coco
import json
import plotly.io as pio
from plotly.subplots import make_subplots

cc = coco.CountryConverter()

df_beer_export_sim = pd.read_pickle('df_beer_export_sim.pkl') #period, user_location = f(country)
brew_country_list = pd.read_pickle('df_brew_country_list.pkl').index.to_list() # list of 204 country
beer_style_list = pd.read_pickle('df_beer_style_list.pkl').to_list() # List of 22 beer style
df_country_style = pd.read_pickle('res.pkl') # Cross tab country style
df_scaling = pd.read_pickle('df_scaling.pkl') #period, user_location = f(style)
df_damso = pd.read_pickle("the_sexiest_porthos.pkl")

df_scaling.drop('Sudan', level=1, axis=0, inplace=True) #J'enlève le Sudan ATTENTION


years = np.arange(2004,2018)
iso_code = cc.pandas_convert(series=pd.Series(brew_country_list), to='ISO3', not_found=None)

In [ ]:
# List of the number of producted beers for each year

app.layout = html.Div([
    html.H2('Brewery succes simulation'),
    html.P("Select a country where you construct your brewery:"),
    dcc.Dropdown(brew_country_list, 'Switzerland', id='country-dropdown'),
    html.P("Select a type of beer that you produce in your brewery:"),
    dcc.Dropdown(beer_style_list, 'Ale', id='type-dropdown'),
    html.Div(id='output-container'),
    dcc.Graph(id="graph")
])

In [5]:
# Generate dropdown menu for country :
for i, c in enumerate(brew_country_list):
    
    print(f'<option value="{iso_code[i]}">{c}</option>')

<option value="USA">United States</option>
<option value="GBR">United Kingdom</option>
<option value="DEU">Germany</option>
<option value="CAN">Canada</option>
<option value="ITA">Italy</option>
<option value="FRA">France</option>
<option value="ESP">Spain</option>
<option value="AUS">Australia</option>
<option value="NLD">Netherlands</option>
<option value="CHE">Switzerland</option>
<option value="BEL">Belgium</option>
<option value="RUS">Russia</option>
<option value="CZE">Czech Republic</option>
<option value="BRA">Brazil</option>
<option value="AUT">Austria</option>
<option value="JPN">Japan</option>
<option value="SWE">Sweden</option>
<option value="POL">Poland</option>
<option value="MEX">Mexico</option>
<option value="DNK">Denmark</option>
<option value="ARG">Argentina</option>
<option value="NZL">New Zealand</option>
<option value="NOR">Norway</option>
<option value="ZAF">South Africa</option>
<option value="CHN">China</option>
<option value="IRL">Ireland</option>
<option value

In [7]:
for b in beer_style_list:
    
    print(f'<option value="{b}">{b}</option>')

<option value="Ale">Ale</option>
<option value="Amber Ale">Amber Ale</option>
<option value="Amber Lager">Amber Lager</option>
<option value="Belgian Ale">Belgian Ale</option>
<option value="Bitter Ale">Bitter Ale</option>
<option value="Bock">Bock</option>
<option value="Brown Ale">Brown Ale</option>
<option value="Cider">Cider</option>
<option value="Dark Ale">Dark Ale</option>
<option value="Dark Lager">Dark Lager</option>
<option value="IIPA">IIPA</option>
<option value="IPA">IPA</option>
<option value="Lager">Lager</option>
<option value="Mead">Mead</option>
<option value="Pale Ale">Pale Ale</option>
<option value="Pilsener">Pilsener</option>
<option value="Porter">Porter</option>
<option value="Saké">Saké</option>
<option value="Sour">Sour</option>
<option value="Specialty Beer">Specialty Beer</option>
<option value="Stout">Stout</option>
<option value="Strong Ale">Strong Ale</option>
<option value="Wheat Beer">Wheat Beer</option>


In [7]:
projection_types = ['natural earth', 'equirectangular', 'orthographic', 'mercator']

# Layout for the figures
layout4 = go.Layout(
    height=500, width=900,
    title='Cumulative results',
    plot_bgcolor='rgba(0,0,0,0)',
    hovermode='x unified',

    yaxis = dict(
      gridcolor = '#909090',
      ticksuffix=' ', 
      showline=True, linewidth=1, linecolor='black', mirror=True,
    ),
    xaxis = dict(
      gridcolor='#909090',
      showline=True, linewidth=1, linecolor='black', mirror=True,
      title = {'text':'year'})  
)
buttons = []
for projection_type in projection_types:
    buttons.append(dict(
                label=projection_type,
                method='relayout',
                args=[{'geo': {'projection': {'type': projection_type}}}]
                ))
    
updatemenus = list([dict(
        buttons=buttons,
        type='dropdown',
        direction='down',
        pad={'r': 10, 't': 10},
        showactive=True,
        x=0,
        xanchor='left',
        y=1.1,
        yanchor='top'
)])

In [10]:
def simulation(brew_country='France', beer_type='Ale', projection='natural earth'):
    # Dataframe to store the exportation distribution in each user countries
    df_distribution = pd.DataFrame(columns=['brew_country','country', 'period', 'value'])
    
    NB_INIT_BEER = 1000
    
    beer_producted_per_year    = np.zeros(len(years)+1, int)
    beer_producted_per_year[0] = NB_INIT_BEER
    
    # Dataframe to store cumulated country where there is distributed beer (False : no distrib, True : distrib)
    country_reach_by_export = [False]*len(df_beer_export_sim.xs(2010).index) #2010 is a random id to have the size of the df
    nb_export_country       = []
    nb_cumul_beers          = []
    
    for i, year in enumerate(years[:-1]):

        #D'Artagnan's part
        beer_distribution = (beer_producted_per_year[i]*df_beer_export_sim.xs(year)[brew_country]).apply(np.ceil)
        
        #Athos part
        if year == 2017:
            beer_distribution = beer_distribution.mul(df_scaling.xs(2016)[beer_type]).mul(1 + df_damso[beer_type].loc[:,2016]).fillna(0).apply(np.ceil)
        else:
            beer_distribution = beer_distribution.mul(df_scaling.xs(year)[beer_type]).mul(1 + df_damso[beer_type].loc[:,year]).fillna(0).apply(np.ceil)

        if beer_distribution.sum() == 0:
            beer_producted_per_year[i+1] = beer_producted_per_year[i]
        else:
            beer_producted_per_year[i+1] = beer_distribution.sum()
        
        if i>0:
            nb_cumul_beers.append(nb_cumul_beers[-1] + beer_distribution.sum())
        else:
            nb_cumul_beers.append(beer_distribution.sum())

        # store this data in df_distribution (beer_distribution_store just used for manipulating data)
        beer_distribution = beer_distribution.reset_index()
        beer_distribution.columns = ['country', 'distribued beer']
        beer_distribution['year'] = str(year)
        beer_distribution['brew_country'] = brew_country
        df_distribution = pd.concat([df_distribution,beer_distribution], axis=0)
        
        # Count the progression of country reached by export
        country_reach_by_export = country_reach_by_export | (beer_distribution['distribued beer'] > 0)
        nb_export_country.append(np.sum(country_reach_by_export))
        
    
    # iso_code for the graph world map
    df_distribution['iso_code'] = cc.pandas_convert(series=df_distribution.country, to='ISO3', not_found=None)
    df_distribution['happiness gauge'] = np.log10(df_distribution['distribued beer'].replace(to_replace = 0, value = 1)).round(2)
    df_distribution
    
    # Select the data for the input country, filter country with no data
    mask = (df_distribution.brew_country == brew_country) & (df_distribution['happiness gauge'] > 0)
    beer_map = px.choropleth(df_distribution[mask], locations="iso_code",
                     color="happiness gauge",
                     hover_name="country",
                     animation_frame="year",
                     title = f"Distribution brewery in {brew_country} producing {beer_type}-type beers",                 
                     color_continuous_scale=px.colors.sequential.Oranges,
                     width=900, height=700, range_color = [0, df_distribution['happiness gauge'].max()],
                     hover_data={'iso_code':False,'year':False, 'distribued beer':':.0f', 'happiness gauge':':.2f'},
                     )
    # Replace the updatemenu
    beer_map["layout"].pop("updatemenus")
    beer_map.update_layout(margin={"r":0,"t":3,"l":0,"b":0}, 
                           title={'xanchor': 'center', 'yanchor': 'top', 'y':0.98, 'x':0.5},
                           updatemenus=updatemenus)
    
    # Set the projection
    beer_map.update_geos(projection_type=projection)
    
    # fig 4
    
    fig = make_subplots(specs=[[{"secondary_y": True}]])
    
    fig.add_trace(
        go.Scatter(x    = years, 
                   y    = nb_export_country,
                   line = go.scatter.Line(color='#B51F1F', width=2),
                   name = "number of country"),
        secondary_y=False
    )
    
    fig.add_trace(
        go.Scatter(x    = years, 
                   y    = nb_cumul_beers,
                   line = go.scatter.Line(color='#636EFA', width=2),
                   name = "number of produced beers"),
        secondary_y=True
    )
    
    # Set x-axis title
    fig.update_xaxes(title_text="year")

    # Set y-axes titles
    fig.update_yaxes(title_text="number of country", secondary_y=False)
    fig.update_yaxes(range = [0,np.max(nb_export_country)+2], secondary_y=False)
    fig.update_yaxes(title_text="number of beers", tickprefix=' ', secondary_y=True)
    
    fig.update_layout(layout4)
    

    return beer_map, fig

In [11]:
f1, f2 = simulation()
f1.show()
f2.show()

In [14]:
figure1_list = '{'
figure2_list = '{'
figure3_list = '{'

for i, c in enumerate(brew_country_list):
    dico1 = '{'
    dico2 = '{'
    #dico3 = '{'
    for s in beer_style_list:
        fig1, fig2 = simulation(c,s)
        dico1 += '"{}" : {},'.format(s, fig1.to_json())
        dico2 += '"{}" : {},'.format(s, fig2.to_json())
        #dico3 += '"{}" : {},'.format(s, fig3.to_json())
        #dico[s] = json.loads(fig_json)
        
    dico1 = dico1[:-1] + '}'
    dico2 = dico2[:-1] + '}'
    #dico3 = dico3[:-1] + '}'
    
    figure1_list += '"{}" : {},'.format(iso_code[i], dico1)
    figure2_list += '"{}" : {},'.format(iso_code[i], dico2)
    #figure3_list += '"{}" : {},'.format(iso_code[i], dico3)
    #figure_list[iso_code[i]] = dico
    
figure1_list = figure1_list[:-1] + '}'
figure2_list = figure2_list[:-1] + '}'
#figure3_list = figure3_list[:-1] + '}'

In [15]:
# Writing to sample.json
with open("beermap.json", "w", encoding='utf-8') as outfile:
    outfile.write("beermap_data = '" + figure1_list + "'")

print("beermap.json written !")

with open("exportfig.json", "w",encoding='utf-8') as outfile:
    outfile.write("exportfig_data = '" + figure2_list + "'")
    
print("exportfig.json written !")
    
#with open("countryfig.json", "w", encoding='utf-8') as outfile:
#    outfile.write("countryfig_data = '" + figure3_list + "'")
#    
#print("countryfig.json written !")

beermap.json written !
exportfig.json written !


In [22]:
f = open('beermap.json')
  
# returns JSON object as 
# a dictionary
data = json.load(f)
  
# Closing file
f.close()

print(data)

{'data': [{'coloraxis': 'coloraxis', 'geo': 'geo', 'hovertemplate': '<b>%{hovertext}</b><br><br>period=2003<br>iso_code=%{location}<br>score=%{z}<extra></extra>', 'hovertext': ['Afghanistan', 'Albania', 'Algeria', 'American Samoa', 'Andorra', 'Angola', 'Anguilla', 'Antarctica', 'Antigua and Barbuda', 'Argentina', 'Armenia', 'Aruba', 'Australia', 'Austria', 'Azerbaijan', 'Bahamas', 'Bangladesh', 'Barbados', 'Belarus', 'Belgium', 'Belize', 'Bermuda', 'Bhutan', 'Bolivia', 'Bosnia and Herzegovina', 'Botswana', 'Bouvet Island', 'Brazil', 'Bulgaria', 'Burkina Faso', 'Burundi', 'Cambodia', 'Canada', 'Cayman Islands', 'Chile', 'China', 'Colombia', 'Congo', 'Costa Rica', 'Croatia', 'Cuba', 'Cyprus', 'Czech Republic', 'Denmark', 'Dominican Republic', 'East Timor', 'Ecuador', 'Egypt', 'El Salvador', 'Equatorial Guinea', 'Estonia', 'Ethiopia', 'Falkland Islands', 'Faroe Islands', 'Fiji Islands', 'Finland', 'France', 'French Guiana', 'Georgia', 'Germany', 'Ghana', 'Gibraltar', 'Greece', 'Greenland'

In [13]:
f1.to_json()

'{"data":[{"coloraxis":"coloraxis","customdata":[["AGO","2004",2.0,0.3],["AUS","2004",20.0,1.3],["BEL","2004",12.0,1.08],["BOL","2004",2.0,0.3],["CAN","2004",138.0,2.14],["HRV","2004",15.0,1.18],["CZE","2004",3.0,0.48],["DNK","2004",93.0,1.97],["FIN","2004",10.0,1.0],["FRA","2004",3.0,0.48],["DEU","2004",15.0,1.18],["ISL","2004",2.0,0.3],["ITA","2004",19.0,1.28],["JPN","2004",7.0,0.85],["NLD","2004",28.0,1.45],["NOR","2004",2.0,0.3],["POL","2004",2.0,0.3],["KOR","2004",2.0,0.3],["ESP","2004",5.0,0.7],["SWE","2004",24.0,1.38],["CHE","2004",27.0,1.43],["TUR","2004",2.0,0.3],["GBR","2004",130.0,2.11],["USA","2004",445.0,2.65]],"geo":"geo","hovertemplate":"<b>%{hovertext}</b><br><br>distribued beer=%{customdata[2]:.0f}<br>happiness gauge=%{z:.2f}<extra></extra>","hovertext":["Angola","Australia","Belgium","Bolivia","Canada","Croatia","Czech Republic","Denmark","Finland","France","Germany","Iceland","Italy","Japan","Netherlands","Norway","Poland","South Korea","Spain","Sweden","Switzerland"